In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from time import time
from sklearn.model_selection import train_test_split
import re
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
df_train = pd.read_csv('../input//train.csv')
df_test = pd.read_csv('../input//test.csv')

In [ ]:
sw = stopwords.words('english')
for i in sw:
    df_train['new_name'] = df_train['question_text'].replace(i, '')
pattern = r"\d+."
df_train['new_name'] = df_train['new_name'].str.replace(pattern, '')
df_head_train = df_train.head(n=5)

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(df_train['new_name'], df_train['target'], test_size=0.3, random_state=42)
vectorizer = TfidfVectorizer(sublinear_tf=False)
features_train_transformed = vectorizer.fit_transform(features_train)
features_test_transformed = vectorizer.transform(features_test)

In [ ]:
n_components = 100
pca = TruncatedSVD(n_components=n_components, random_state=42, n_iter=10).fit(features_train_transformed)

X_train_pca = pca.transform(features_train_transformed)
X_test_pca = pca.transform(features_test_transformed)

In [ ]:
clf = ExtraTreesClassifier(n_estimators=150, criterion='gini',random_state=42)
clf.fit(X_train_pca, labels_train)
predicted_values_0 = clf.predict(X_test_pca)
accuracy = accuracy_score(labels_test, predicted_values_0)
features_test_transformed_submission = vectorizer.transform(df_test.question_text)
X_test_pca_submission = pca.transform(features_test_transformed_submission)
predicted_values_for_submission = clf.predict(X_test_pca_submission)

In [ ]:
submisiion = pd.DataFrame({'qid':df_test.qid, 'prediction': predicted_values_for_submission}, columns=['qid', 'prediction'])
submisiion.to_csv('submission.csv', index=False)